In [1]:
import pandas as pd
import session_info

session_info.show()

## Read in data

### MS Teams spreadsheet grades

In [2]:
spreadsheet = pd.read_csv(
    # 'data/apsc160-2021w1-grades-2022edition.csv'
    'data/apsc160-2021w1-grades-feb09.csv'
).rename(
    columns = lambda x: x.lower()
).rename(
    columns={'snum': 'student_number'}
).assign(
    name=lambda df: df['pref'] + ' ' + df['last'],
).sort_values(
    'name'
)[['student_number', 'cwl', 'name']]#, 'Name_first']]
spreadsheet

,student_number,cwl,name
6,63728547,akanagal,Aakash Kanagala
321,82779430,aal,Aaron Lee
323,37388220,aarony03,Aaron Luo
319,47241583,NaN,Aaron Wu
0,22082580,aaryan6,Aaryan Patel
...,...,...,...
318,37538964,znj03,Zeena Jarallah
371,50333350,bwz,Zhe Wang
591,15740723,zoescy,Zoe So
592,99569139,zoeyuen,Zoe Yuen


### PrairieLearn

File downloaded via `Gradebook -> Download`.

In [3]:
pl = pd.read_csv(
    'data/2022-01-25T1032_Grades-APSC_160_2021W1.csv'
).drop(
    index=[0,1]  # empty rows
).rename(
    columns = lambda x: x.lower()
).rename(
    columns={'sis user id': 'student_number'}
).assign(
    name=lambda df: df['student'].str.split(', ').apply(reversed).str.join(' '),
).query(
    'name != "Test student"'
).sort_values(
    'name'
)[['student_number', 'name']]
pl

,student_number,name
226,63728547.0,Aakash Kanagala
273,82779430.0,Aaron Lee
307,37388220.0,Aaron Luo
553,47241583.0,Aaron Wu
376,22082580.0,Aaryan Patel
...,...,...
216,37538964.0,Zeena Jarallah
538,50333350.0,Zhe Wang
463,15740723.0,Zoe So
587,99569139.0,Zoe Yuen


### Canvas

File downloaded via `Grades -> Actions -> Export`.

In [4]:
canvas = pd.read_csv(
    'data/APSC_160_2021W1_gradebook.csv'
).rename(
    columns = lambda x: x.lower()
).query(
    'role == "Student"'
).assign(
    cwl=lambda df: df['uid'].str.split('@', expand=True)[0]
).sort_values(
    'name'
)[['cwl', 'name']]
canvas

,cwl,name
21,akanagal,Aakash Kanagala
2,aal,Aaron Lee
3,aarony03,Aaron Luo
4,aaryan6,Aaryan Patel
15,afaouri,Abdel-Rahman AL-Faouri
...,...,...
110,bwz,Zhe Wang
645,zoesimms,Zoe Simmons
644,zoescy,Zoe So
646,zoeyuen,Zoe Yuen


## Mismatched student names between the spreadsheet and PL

In [5]:
spreadsheet_pl = spreadsheet.merge(
    pl,
    on='name',
    how='outer',
    indicator=True,
    suffixes=('__spreadsheet', '__pl')
)
spreadsheet_pl[spreadsheet_pl.isna().any(axis=1)]

,student_number__spreadsheet,cwl,name,student_number__pl,_merge
3,47241583.0,NaN,Aaron Wu,47241583.0,both
9,38643375.0,NaN,Adham Ahmed,38643375.0,both
83,56711658.0,ben9354,Ben Chung,NaN,left_only
337,41689902.0,vks28903,Kiara Suyanto,NaN,left_only
369,79800975.0,lucaspri,Lucas Rimini,NaN,left_only
416,33752544.0,elhan31,Mohammad Elhan Iqbal,NaN,left_only
448,65584427.0,pauglz03,Paulina GonzÃ¡lez,NaN,left_only
586,74579665.0,zakv,Zachary Varieur,NaN,left_only
594,NaN,NaN,Benjamin Chung,56711658.0,right_only
595,NaN,NaN,Elhan Iqbal,33752544.0,right_only


All mismatches are seemingly due to different spelling of the same student names.

### Fix the mismatched student names and merge again

In [6]:
pl = pl.assign(
    name=lambda df: df['name'].replace({
        'Benjamin Chung': 'Ben Chung',
        'Elhan Iqbal': 'Mohammad Elhan Iqbal',
        'Lucas Pavel Rimini': 'Lucas Rimini',
        'Paulina González': 'Paulina GonzÃ¡lez',
        'Vincentia Kiara Suyanto': 'Kiara Suyanto',
        'Zak Varieur': 'Zachary Varieur'
    })
)

spreadsheet_pl = spreadsheet.merge(
    pl,
    on='name',
    how='outer',
    indicator=True,
    suffixes=('__spreadsheet', '__pl')
)
spreadsheet_pl[spreadsheet_pl.isna().any(axis=1)]

,student_number__spreadsheet,cwl,name,student_number__pl,_merge
3,47241583,NaN,Aaron Wu,47241583.0,both
9,38643375,NaN,Adham Ahmed,38643375.0,both


We looked into these two students and they do not have any scores on PL,
so it is fine that they are missing CWLs.

## Mismatched student names between the combined spreadsheet/PL frame and Canvas

In [7]:
spreadsheet_pl_canvas = spreadsheet_pl.drop(
    columns='_merge'  # since we are now recreating this column
).merge(
    canvas,
    on='name',
    how='outer',
    indicator=True,
    suffixes=('__spreadsheet', '__canvas')
)
spreadsheet_pl_canvas[spreadsheet_pl_canvas.isna().any(axis=1)]

,student_number__spreadsheet,cwl__spreadsheet,name,student_number__pl,cwl__canvas,_merge
3,47241583.0,NaN,Aaron Wu,47241583.0,NaN,left_only
9,38643375.0,NaN,Adham Ahmed,38643375.0,NaN,left_only
570,51608354.0,ychahbaz,Yara Chahbaz,51608354.0,NaN,left_only
594,NaN,NaN,Ada Hmoud,NaN,fhmoud,right_only
595,NaN,NaN,Aldiyar Mukatay,NaN,amukatay,right_only
596,NaN,NaN,Alexandre Jennison,NaN,ajenn,right_only
597,NaN,NaN,Allya Wellyanto,NaN,allwelly,right_only
598,NaN,NaN,Amelia Savazzi,NaN,asavazzi,right_only
599,NaN,NaN,Amr Sherif,NaN,amr2021,right_only
600,NaN,NaN,Aren Dexter-Nickel,NaN,adn,right_only


I looked into the student Yara Chahbaz who has a CWL in the spreadsheet but not on Canvas,
and they don't have any scores on PL,
so I don't think we need to worry about them.
All the other students appear on Canvas but not on PL,
which I believe means that they have dropped the course at some point
so I am dropping them in the next cell.

In [8]:
spreadsheet_pl_canvas = spreadsheet_pl_canvas.query('_merge != "right_only"')

## Differences between student numbers in the spreadsheet and on PL

In [9]:
spreadsheet_pl_canvas[spreadsheet_pl_canvas['student_number__spreadsheet'] != spreadsheet_pl_canvas['student_number__pl']]

,student_number__spreadsheet,cwl__spreadsheet,name,student_number__pl,cwl__canvas,_merge


All students have the same student numbers in the spreadsheet as they do on PL.

## Differences between CWLs in the spreadsheet and on Canvas

In [10]:
spreadsheet_pl_canvas[spreadsheet_pl_canvas['cwl__spreadsheet'] != spreadsheet_pl_canvas['cwl__canvas']]

,student_number__spreadsheet,cwl__spreadsheet,name,student_number__pl,cwl__canvas,_merge
3,47241583.0,NaN,Aaron Wu,47241583.0,NaN,left_only
9,38643375.0,NaN,Adham Ahmed,38643375.0,NaN,left_only
541,96454657.0,tilsu,Til Schmitt-Ulms,96454657.0,tilsu262,both
570,51608354.0,ychahbaz,Yara Chahbaz,51608354.0,NaN,left_only


There is one student "Til Schmitt-Ulms"
who has a different CWL in the spreadsheet (`tilsu`)
compared to on Canvas (`tilsu262`).
We should double check so that this student's grade is still uploaded correctly
to the FSC.